# Tutorial 1 Appendix

Version: 1.2 (January 2018)

This appendix prodivdes the code for the preparation of the data sets used in Tutorial 1. The general steps are as follows:

1. Retrieve a reference *Escherichia coli* genome assembly via FTP from NCBI, filter out to only the terms we want
2. Download and select two random genomes from a list (we have ~30 students, so 9 genomes should suffice for unique pairs).
3. BLAST the genes from 1. against the genes from the genomes presented in 2, returning any strong hits.
4. Write the strongest hit to a file

This was written and run on Windows (as a challenge to myself), but it should work anywhere.

In [1]:
# Change to False if you want to actually run the data-prep code
# This is just to mask some code when we want to import the functions
# for the assignment.
just_defs = True

In [2]:
import os
import subprocess
import gzip
from ftplib import FTP
from itertools import combinations

if not just_defs:
    from Bio.Blast.Applications import NcbiblastpCommandline
    from Bio.Blast import NCBIXML

In [3]:
student_ids = ["B00759975", "B00792164", "B00712385", "B00083078", 
               "B00728282", "B00780317", "B00782621", "B00672276", 
               "B00423833", "B00514032", "B00684615", "B00784897", 
               "B00592569", "B00760708", "B00786252", "B00759971", 
               "B00748375", "B00775690", "B00194191", "B00487044", 
               "B00735030", "B00774756", "B00555880", "B00755962", 
               "B00612172", "B00769297", "B00536396", "B00774740", 
               "B00475892", "B00786366", "B00559291", "B00698075"]
reference_genome = "GCA_000005845" #Escherichia coli K-12 strain MG1655
genomes = ["GCA_001553935", #Actinomyces oris
           "GCA_002734145", #Faecalibacterium prausnitzii
           "GCA_000008805", #Neisseria meningitidis
           "GCA_000008625", #Aquifex aeolicus
           "GCA_000513215", #Serratia marcescens
           "GCA_000008565", #Deinococcus radiodurans
           "GCA_000237865", #Haloquadratum walsbyi
           "GCA_000238215", #Tannerella forsythia
           "GCA_000011805"] #Vibrio fischeri

In [4]:
# Assign a genome pair to each student, providing a function that they can use to get their filenames
def get_genome_pair(student_id):
    genome_pairs = list(combinations(genomes, 2))
    try:
        idx = student_ids.index(student_id)
        return genome_pairs[idx]
    except ValueError:
        print("Error: Student ID not in list. Is it correct?")
        raise
#Example, showing my genome pair
if not just_defs:
    get_genome_pair('B00698075')

In [5]:
if not just_defs:
    os.chdir("tutorial1_appendix_data")

# 1. Download Reference Genome

In [6]:
def fetch_protein_set(genome_id, out_file):
    filenames = []
    def get_name(line):
        filenames.append(line.split(" ")[-1])
    genome_split = genome_id.split("_")
    ftp = FTP("ftp.ncbi.nlm.nih.gov")
    ftp.login(user='anonymous',passwd='')
    ftp.cwd("genomes/all/")
    # Build most of the URL to the genome on the FTP server
    genome_uri = genome_split[0] + "/" + genome_split[1][0:3] + "/" + genome_split[1][3:6] + "/" + genome_split[1][6:9]
    ftp.cwd(genome_uri)
    # Request the revisions
    code = ftp.retrlines("LIST", callback=get_name)
    # Take the most recent genome revision
    genome_revision = filenames[-1]
    ftp.cwd(genome_revision)
    # Download it
    ftp.retrbinary('RETR %s_protein.faa.gz' % (genome_revision,), open(out_file, 'wb').write)

In [7]:
if not just_defs:
    fetch_protein_set(reference_genome, "reference_proteins.faa.gz")

In [8]:
def filter_genome(in_file, out_file):
    # Filter the .faa file to only keep the items with 'ribosome', 'translation', or 'tRNA' in the description
    seq = False
    with open(out_file, 'w') as subset_file:
        for line in gzip.open(in_file,'rt'):
            line = str(line)
            if (line[0] == ">"):
                if any(x in line.lower() for x in ['ribosome','trna','translation','ribosomal']):
                    seq = True
                    subset_file.write(line)
                else:
                    seq = False
            else:
                if seq:
                    subset_file.write(line)

In [9]:
if not just_defs:
    filter_genome("reference_proteins.faa.gz", "subset_reference.faa")

## 2. Download Other Genomes

In [10]:
# Just a simple little gunzipper
def decompress_file(in_file, out_file):
    with gzip.open(in_file, 'rt') as f:
        with open(out_file, 'w') as o:
            for line in f:
                o.write(str(line))

In [11]:
if not just_defs:
    for genome_id in genomes:
        fetch_protein_set(genome_id, genome_id + ".faa.gz")
        decompress_file(genome_id + ".faa.gz", genome_id + ".faa")
        #Format a BLAST database for each one
        #Windows: makeblastdb.exe
        #UNIX/MacOS: makeblastdb
        subprocess.run("makeblastdb.exe -in %s.faa -dbtype prot -out %s" % (genome_id, genome_id))

## 3. BLAST Genomes to Reference

In [12]:
if not just_defs:
    #For each of the genomes, blast the reference subset to find the closest protein match
    #Store the results in XML filesTb
    for genome_id in genomes:
        blast_res = NcbiblastpCommandline(cmd="blastp", query="subset_reference.faa", db=genome_id,
                                          evalue=1e-10, outfmt=5, out="reference_%s.xml" % (genome_id,))
        blast_res()

## 4. Filter Genomes to Best Hits Against Reference

In [13]:
if not just_defs:
    # Iterate through BLAST results, pulling the accessions from the best hits for each sequence, 
    # if any hit above the e-value threshold exists
    for genome_id in genomes:
        result_handle = open("reference_%s.xml" % (genome_id,))
        blast_records = NCBIXML.parse(result_handle)
        best_hit_labels = []
        skipped = 0
        for blast_record in blast_records:
            if blast_record.alignments:
                # Add the accession
                best_hit_labels.append(blast_record.alignments[0].title.split(" ")[1])
            else:
                skipped += 1
        result_handle.close()
        
        # Now loop through the original faa files and pull out the right sequences
        seq = False
        # Note that one of the E. coli genes might hit more than one of the same gene from a genome
        # Meaning that you may end up with fewer genes than the reference set
        with open(genome_id + "_subset.faa", 'w') as subset_file:
            with open(genome_id + ".faa", 'r') as fasta_file:
                for line in fasta_file:
                    if (line[0] == ">"):
                        if any(x in line.strip() for x in best_hit_labels):
                            seq = True
                            subset_file.write(line)
                        else:
                            seq = False
                    else:
                        if seq:
                            subset_file.write(line)